# Main Agent - Amadeus GDS Orchestrator

This notebook implements the main orchestrator agent that coordinates the entire workflow:
1. Receives user query from Slack handler
2. Calls query_agent to interact with Amadeus API
3. Calls explainer_agent to format the response
4. Returns human-friendly response for Slack

## Technology:
- OpenAI Agents SDK
- Tracing enabled for debugging and monitoring

## Integration:
The `process_user_query()` function is the entry point called by Slack_agent.ipynb

## Section 1: Setup & Dependencies

Install and import required packages, load environment variables

In [ ]:
# Install required packages
!pip install openai-agents-sdk python-dotenv nest-asyncio

In [5]:
import os
import asyncio
from dotenv import load_dotenv
from openai import OpenAI
from agents import Agent, Runner, trace

# Load environment variables
load_dotenv(override=True)

# Initialize OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

print("✅ OpenAI client initialized")
print("✅ Tracing will be used in workflow execution")

✅ OpenAI client initialized
✅ Tracing will be used in workflow execution


## Section 2: Tool Definitions (Placeholders)

Define placeholder tools for query_agent and explainer_agent.
These will be replaced with actual agent implementations later.

In [6]:
def query_agent_tool(user_query: str) -> dict:
    """
    Placeholder for query_agent - will interact with Amadeus API.
    
    Args:
        user_query: Natural language query from user
        
    Returns:
        dict: Structured flight data from Amadeus API
    """
    # TODO: Replace with actual query_agent implementation
    print(f"🔍 Query Agent called with: {user_query}")
    
    # Mock response simulating Amadeus API data
    return {
        "status": "success",
        "query": user_query,
        "flights": [
            {
                "airline": "American Airlines",
                "flight_number": "AA123",
                "origin": "JFK",
                "destination": "LAX",
                "departure_time": "08:00 AM",
                "arrival_time": "11:30 AM",
                "duration": "5h 30m",
                "price": "$299",
                "stops": "Non-stop"
            },
            {
                "airline": "Delta",
                "flight_number": "DL456",
                "origin": "JFK",
                "destination": "LAX",
                "departure_time": "02:00 PM",
                "arrival_time": "05:45 PM",
                "duration": "5h 45m",
                "price": "$325",
                "stops": "Non-stop"
            }
        ]
    }


def explainer_agent_tool(flight_data: dict) -> str:
    """
    Placeholder for explainer_agent - formats API response into human-readable text.
    
    Args:
        flight_data: Raw Amadeus API response data
        
    Returns:
        str: Formatted, human-friendly response
    """
    # TODO: Replace with actual explainer_agent implementation
    print(f"📝 Explainer Agent called with flight data")
    
    # Mock formatted response
    flights = flight_data.get("flights", [])
    
    response = f"I found {len(flights)} flights for your search:\n\n"
    
    for i, flight in enumerate(flights, 1):
        response += f"✈️ Option {i}: {flight['airline']} {flight['flight_number']}\n"
        response += f"   • Route: {flight['origin']} → {flight['destination']}\n"
        response += f"   • Departure: {flight['departure_time']} | Arrival: {flight['arrival_time']}\n"
        response += f"   • Duration: {flight['duration']} ({flight['stops']})\n"
        response += f"   • Price: {flight['price']}\n\n"
    
    return response


print("✅ Placeholder tools defined: query_agent_tool, explainer_agent_tool")

✅ Placeholder tools defined: query_agent_tool, explainer_agent_tool


## Section 3: Main Agent Definition

Create the orchestrator agent with clear instructions and tool access

In [7]:
# Define the main orchestrator agent
main_agent = Agent(
    name="Amadeus GDS Orchestrator",
    instructions="""
    You are the main orchestrator for the Amadeus GDS Helper system.
    
    Your role:
    1. Receive user queries about flights (search, booking, modifications, cancellations)
    2. Understand the user's intent from their natural language query
    3. Call the query_agent_tool to get flight data from Amadeus API
    4. Call the explainer_agent_tool to format the API response into human-readable text
    5. Return the formatted response to the user
    
    Key principles:
    - Always call query_agent_tool first to get the data
    - Then call explainer_agent_tool to format the response
    - Be helpful, concise, and user-friendly
    - Handle errors gracefully and inform the user clearly
    
    The workflow is: User Query → Query Agent → Explainer Agent → Formatted Response
    """,
    model="gpt-4o",
    tools=[query_agent_tool, explainer_agent_tool]
)

print("✅ Main agent created: Amadeus GDS Orchestrator")
print("📋 Tools available: query_agent_tool, explainer_agent_tool")

✅ Main agent created: Amadeus GDS Orchestrator
📋 Tools available: query_agent_tool, explainer_agent_tool


<cell_type>markdown</cell_type>## Section 4: Runner Configuration

The Runner is used via `Runner.run()` as a static method (no instantiation needed)

In [8]:
# No runner instantiation needed - Runner.run() is a static method
print("✅ Runner will be used via Runner.run() static method")
print("✅ Agent workflow is ready to execute")

✅ Runner will be used via Runner.run() static method
✅ Agent workflow is ready to execute


## Section 5: Process User Query Function

Main entry point for Slack integration - processes user queries and returns responses

In [ ]:
async def process_user_query_async(user_query: str) -> str:
    """
    Process a user query through the main agent workflow (async version).
    
    This function is called by the Slack handler and orchestrates:
    1. Query agent to get Amadeus API data
    2. Explainer agent to format the response
    
    Args:
        user_query: Natural language query from Slack user
        
    Returns:
        str: Formatted response ready for Slack posting
    """
    try:
        print(f"\n{'='*60}")
        print(f"🚀 Processing query: {user_query}")
        print(f"{'='*60}\n")
        
        # Run the agent workflow with tracing
        with trace("main_agent_workflow", metadata={"query": user_query}) as t:
            result = await Runner.run(main_agent, user_query)
        
        # Extract the final response
        response = result.final_output
        
        print(f"\n{'='*60}")
        print(f"✅ Workflow completed successfully")
        print(f"{'='*60}\n")
        
        return response
        
    except Exception as e:
        error_message = f"❌ Error processing query: {str(e)}"
        print(f"\n{error_message}\n")
        return f"Sorry, I encountered an error while processing your request: {str(e)}\nPlease try again or contact support."


def process_user_query(user_query: str) -> str:
    """
    Synchronous wrapper for process_user_query_async.
    This is the main entry point called by Slack handler.
    
    Args:
        user_query: Natural language query from Slack user
        
    Returns:
        str: Formatted response ready for Slack posting
    """
    # Check if we're in a Jupyter notebook with an existing event loop
    try:
        loop = asyncio.get_running_loop()
        # If we're here, there's already a loop running (Jupyter)
        # Create a task and run it
        import nest_asyncio
        nest_asyncio.apply()
        return asyncio.run(process_user_query_async(user_query))
    except RuntimeError:
        # No event loop running (normal Python script)
        return asyncio.run(process_user_query_async(user_query))


print("✅ process_user_query() function ready (sync wrapper)")
print("✅ process_user_query_async() function ready (async implementation)")
print("📌 Use process_user_query() as the integration point for Slack_agent.ipynb")

<cell_type>markdown</cell_type>## Section 6: Manual Testing

Test the main agent with sample queries

**Note:** In Jupyter notebooks, we can use `await` directly since the event loop is already running.

In [ ]:
# Test Query 1: Flight search
test_query_1 = "Find me flights from New York to Los Angeles next Friday"

# In Jupyter, use await directly
response_1 = await process_user_query_async(test_query_1)
print("\n📬 Response for Slack:")
print(response_1)

In [ ]:
# Test Query 2: Different route
test_query_2 = "Show me available flights from Boston to Miami on December 15th"

# In Jupyter, use await directly
response_2 = await process_user_query_async(test_query_2)
print("\n📬 Response for Slack:")
print(response_2)

In [ ]:
# Test Query 3: General inquiry
test_query_3 = "I need to book a flight from Chicago to San Francisco for a business trip"

# In Jupyter, use await directly
response_3 = await process_user_query_async(test_query_3)
print("\n📬 Response for Slack:")
print(response_3)

## Next Steps

1. ✅ Main agent orchestrator is ready
2. 🔜 Implement actual query_agent (Amadeus API integration)
3. 🔜 Implement actual explainer_agent (response formatting)
4. 🔜 Integrate with Slack_agent.ipynb by importing `process_user_query()`

## Integration with Slack

In `Slack_agent.ipynb`, replace the placeholder response with:

```python
from main_agent import process_user_query

@app.command("/ask_amadeus")
def handle_ask_amadeus_command(ack, command, say):
    ack()
    user_query = command['text']
    
    # Call the main agent
    response = process_user_query(user_query)
    
    # Send to Slack
    say(response)
```